In [ ]:
import sys
sys.path.append(r'C:\Users\Lina\Documents\Lab-I---Dimensionality-Reduction\Unsupervised_model')

from SVD import SVD as MySVD
from PCA import PCA as MyPCA
from TSNE import TSNE as MyTSNE

import json
import numpy as np
from flask import Flask, request
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
mnist = fetch_openml('mnist_784')
mnist.target = mnist.target.astype(int)
X, y = mnist['data'], mnist['target']

In [ ]:
#Reducimos la dimensionalidad de los datos de entrada usando nuestro paquete Python no supervisado:
n_components = 20  # número de componentes para reducción de dimensionalidad
pca = MyPCA(n_components=20)
X_pca_fit = pca.fit(X)
X_pca = pca.fit_transform(X)

Al estandarizar los datos, se asegura que todos los datos de entrada estén en una escala similar y se reduce la probabilidad de que se produzcan valores no finitos durante el entrenamiento del modelo

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_pca)

In [ ]:
#Entrenamos el clasificador de dígitos MNIST
clf = MLPClassifier(hidden_layer_sizes=(50,), max_iter=10, alpha=1e-4, solver='sgd', verbose=10, tol=1e-4, random_state=1, learning_rate_init=.1)
clf.fit(X_train_scaled, y)

In [ ]:
app = Flask(__name__)

@app.route('/mnist', methods=['POST'])
def classify_mnist():
    # extraemos la imagen de la solicitud POST
    data = request.json['data']
    
    # convertimos la imagen en una matriz numpy y la redimensionamos
    X = np.array(data).reshape(1, -1)
    
    # reducimos la dimensionalidad de la imagen usando PCA
    X_pca = pca.transform(X)
    
    # predecimos la clase de la imagen usando el clasificador pre-entrenado
    y_pred = clf.predict(X_pca)
    
    # devolvemos la clase predicha como respuesta JSON
    return json.dumps({'class': y_pred.tolist()[0]})

In [ ]:
if __name__ == '__main__':
    app.run()